# Training a 3D CNN

In [9]:
from faimed3d.basics import *
from faimed3d.augment import *
from faimed3d.models import *
from faimed3d.data import *

import pathlib
import re
from fastai.basics import *

## Loading the data

In [10]:
train = pathlib.Path('../../dl-prostate-mapping/data/train')
valid = pathlib.Path('../../dl-prostate-mapping/data/valid')
test = pathlib.Path('../../dl-prostate-mapping/data/test')

train_files = list(train.rglob('DICOM'))
valid_files = list(valid.rglob('DICOM'))
test_files = list(test.rglob('DICOM'))

# take only T2 and T1 images for noe
subset_train =[]
for f in train_files: 
    m = re.search(r'T2', str(f)) 
    if hasattr(m, 'string'): subset_train.append(Path(m.string))
        
subset_valid =[]
for f in valid_files: 
    m = re.search(r'T2', str(f)) 
    if hasattr(m, 'string'): subset_valid.append(Path(m.string))
        
subset_test = []
for f in test_files: 
    m = re.search(r'T2', str(f)) 
    if hasattr(m, 'string'): subset_test.append(Path(m.string))
        
        
def label_func(fn):
    return re.findall(r'(Gesund|ProstataCa)', str(fn))[0]
labels = ['Gesund', 'ProstataCa']

In [11]:
oversampled_train = subset_train + random.choices(subset_train[slice(0, 33)], k = 23)

## Setting up dataloaders

In [12]:
labels = ['Gesund', 'ProstataCa']
def label_func(fn): return re.findall(r'(Gesund|ProstataCa)', str(fn))[0]

In [13]:
mris = DataBlock(
    blocks = (ImageBlock3D(cls=TensorDicom3D), 
              CategoryBlock),
    get_x = lambda x: x,
    get_y = label_func, 
    item_tfms = ResizeCrop3D(crop_by = (0., 0.1, 0.1), resize_to = (10, 150, 150), perc_crop = True),
    batch_tfms = [*aug_transforms_3d(), RandomCrop3D((0, 25, 25), (0, 25, 25)), PseudoColor],
    splitter = RandomSplitter())

In [17]:
dls = mris.dataloaders(oversampled_train+subset_valid)

In [20]:
roc = RocAucBinary()
def loss_func(out, targ):
    return CrossEntropyLossFlat()(out, targ.long())

In [21]:
learn = Learner(dls, AlexNet_3D(), opt_func = SGD, loss_func = loss_func, metrics = [error_rate, roc])
learn = learn.to_fp16()

In [22]:
learn.lr_find()

AssertionError: Mixed-precision training requires a GPU, remove the call `to_fp16`

In [14]:
learn.fit_one_cycle(n_epoch = 1, lr_max = 0.1)

epoch,train_loss,valid_loss,error_rate,roc_auc_score,time
0,0.707710,0.693286,0.500000,0.605000,03:17


In [ ]:
preds, target = learn.get_preds()

In [ ]:
preds = F.softmax(preds, dim = 1)[:, 1].numpy()

It's usually because your network is not complex enough to find a pattern between your input vectors and your output vectors, and therefore, your last output layer is converging towards the average vector of all the outputs in your dataset.

To overcome this there are a few techniques:

1. Try to do some more preprocessing to your inputs, perhaps a PCA on your attributes.
2. Visualize your layers, try to add random vectors as your input and check the outputs of each layer. There must be just one layer which would be outputting almost the same vector everytime, causing problems for your higher level neurons.
3. Reduce your learning rate.
4. Reduce your batch size.
5. Stack more layers.
6. Check if your model is actually learning : send random noise as your data, and the network loss should not be decreasing.

In [ ]:
from sklearn.metrics import roc_curve
ns_fpr, ns_tpr, _ = roc_curve(target.numpy(), preds)
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
preds

In [ ]:
target

In [ ]:
learn.fit_one_cycle(n_epoch = 100, lr_max = 0.001)

In [ ]:
11520/24